In [40]:
import requests
import json
import csv

In [49]:
#this admin profile is created when the app is first launched (defined in run.py)
user = {'username': 'administrator', 'email': 'placeholder','password': 'Admin10!', 'age': 26, 'activity_level': 2, 'gender_identity': 1}

## Login (user and admin)

In [50]:
r = requests.post('http://127.0.0.1:5000/login', json = user)
data = r.json()
try:
    access = data['access_token']
    refresh = data['refresh_token']
    print(data['message'])
    print('JSON Web Token: ' + access[:30])
except:
    print(data['message'])

Logged in as administrator. (admin)
JSON Web Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUz


# AdminResources

### User info (admin)

In [51]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/user/info', headers=headers)
r.json()

### User preferences (admin)

In [52]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/user/preferences', headers=headers)
r.json()

### Food data (dump) (admin)

In [53]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/food/dump', headers=headers)
r.json()

### Food data (add) (admin)

In [9]:
food = {
'item_name' : 'juice box',
'latitude': 44, 
'longitude': -79, 
'restaurant_name' : 'Local bar',
'item_description' : 'This is secretly a salad.',
'nutrition' : json.dumps({'calories':100,'sugar':50,'total_fats': 120,'sat_fats':40,'cholesterol':150,'carbohydrates':100,'protein':20, 'fiber':30,'sodium':120,'magnesium':100,'potassium':40,'calcium':12}),
'price' : 2.30
}

In [54]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/food/add', headers=headers, json = food)
r.json()

### Food data (edit) (admin)

In [70]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/food/edit', headers=headers, json = food)
r.json()

### Food data (remove) (admin)

In [55]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/food/remove', headers=headers, json = food)
r.json()

### Add a new admin (admin)

In [56]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/access', headers=headers, json = {'new_admin':'joshuadrc'})
r.json()

# UserResources

In [45]:
#test user 1 is joshuadrc, age 26, inactive, male.
user = {'username': 'joshuadrc', 'email': 'jsd42@duke.edu','password': 'Admin10!', 'age': 26, 'activity_level': 0, 'gender_identity': 1}

In [74]:
#test user 2 is juliedrc, age 42, active, female
user = {'username': 'juliedrc', 'email': 'joshuadrc@gmail.com','password': 'Admin10!', 'age': 42, 'activity_level': 2, 'gender_identity': 0}

## Registration (user)

In [57]:
#create user if not already created
r = requests.post('http://127.0.0.1:5000/registration', json = user)
r.json()

## Login (user and admin)

In [58]:
#login user
r = requests.post('http://127.0.0.1:5000/login', json = user)
data = r.json()
try:
    access = data['access_token']
    refresh = data['refresh_token']
    print(data['message'])
    print('JSON Web Token: ' + access[:30])
except:
    print(data['message'])

## Login Refresh (user and admin)

In [59]:
headers = { 'Authorization' : 'Bearer ' + refresh}

r = requests.post('http://127.0.0.1:5000/token/refresh', headers=headers)
data = r.json()
access = data['access_token']
print(data['message'])

## Preferences

### Add preference

In [60]:
headers = { 'Authorization' : 'Bearer ' + access}

data = {'preference': 'juice box', 
        'preference_action': 'add'}

r = requests.post('http://127.0.0.1:5000/editpreferences', json = data, headers = headers)
r.json()

### Remove preference

In [61]:
headers = { 'Authorization' : 'Bearer ' + access}

data = {'preference': 'juice box', 
        'preference_action': 'remove'}

r = requests.post('http://127.0.0.1:5000/editpreferences', json = data, headers = headers)
r.json()

### List preferences

In [62]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.get('http://127.0.0.1:5000/preferences', headers = headers)
r.json()['message']

## Recommendations

In [63]:
headers = { 'Authorization' : 'Bearer ' + access}
data = {'latitude': 44, 'longitude': -79, 'distance': 1000}

r = requests.post('http://127.0.0.1:5000/recommendation', headers = headers, json=data)

try:
    for i in json.loads(r.json()['data']):
        print('-----------------------------------------------------------')
        print('Based on: ' + user['username'] + '\'s preference for ' + i['based_on'])
        print('-----------------------------------------------------------')
        for item in i:
            print(f'{item}: {i[item]} \n')
except: 
    print(r.json())

## Logout

In [64]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/logout/access', headers = headers)
r.json()

# Bulk food add example

In [18]:
headers = { 'Authorization' : 'Bearer ' + access}
with open('Archive/food.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        try:
            a = row[0].split('|')
            toadd = {
                'item_name' : str(a[0]),
                'latitude' : float(a[1]),
                'longitude' : float(a[2]),
                'restaurant_name' : a[3],
                'item_description' : 'food description',
                'price' : 1.50, 
                'nutrition': json.dumps({'calories': 200})
            }

            r = requests.post('http://127.0.0.1:5000/admin/food/add', headers = headers, json = toadd)
        except:
            continue